In [1]:
import yfinance as yf  # Data
import pandas as pd  # DataFrame
import plotly.graph_objects as go  # Graph
import time

In [2]:
# Fix: เก็บข้อมูล asset ใน class
# Fix: อยากปิด copilot มากกกกกกกกกกกกกกกกกกกกกกกกกกก
class Asset:
    pass


class Portfolio:
    def __init__(self, cash, port=None):
        """
        ...
        """
        if port is None:
            port = {}
        self.cash = cash
        self.portfolio = port
        self.assets = [Asset.name]

    def add_asset(self, symbol, prices, volumes):
        self.portfolio[symbol] = {"prices": prices, "volumes": volumes}

    def buy_bitcoin(self, hist, amount, asset):  # amount in dollars
        if amount > self.cash:
            print("Not enough cash to buy Bitcoin")
            return self.cash
        if asset not in self.portfolio:
            self.add_asset(asset, [], [])
        self.portfolio[asset]["prices"].append(hist)  # append price
        self.portfolio[asset]["volumes"].append(float(amount / hist))
        print(f"Bought {amount / hist} BTC at {hist} USD")
        self.cash -= amount
        print(f"Remaining cash: {self.cash:.2f} USD")
        print(f"Updated portfolio: {self.portfolio}")
        return self.cash
        # update cash

    def sell_bitcoin(self, hist, amount, asset):
        """
        amount in bitcoins
        """
        if amount > sum(self.portfolio[asset]["volumes"]):
            print(f"Not enough {asset} to sell")
            return self.cash
        if asset not in self.portfolio:
            print(f"No {asset} to sell")
            return self.cash

        volumes = self.portfolio[asset]["volumes"]
        prices = self.portfolio[asset]["prices"]

        while amount > 0 and volumes:
            if amount >= volumes[0]:
                total_sell = amount * prices[0]
                amount -= volumes[0]
                self.cash += total_sell
                print(
                    f"Sold {self.portfolio[asset]['volumes'][0]} BTC at {prices[0]} USD"
                )
                volumes.pop(0)
                prices.pop(0)
            elif amount < self.portfolio[asset]["volumes"][0]:
                volumes[0] -= amount
                self.cash += amount * prices[0]
                amount = 0
                print(
                    f"Sold {self.portfolio[asset]['volumes'][0]} BTC at {prices[0]} USD"
                )
        if volumes == [] or prices == []:
            self.portfolio.pop(asset)
        # Fix: vvvvvvvvvvvv ไม่ควร print ใน method
        print(f"Remaining cash: {self.cash:.2f} USD")
        print(f"Updated portfolio: {self.portfolio}")
        return self.cash

    def get_total_volume(self):
        """
        Returns the total volume of all assets in the portfolio.
        """
        pass

    def get_asset(self):
        """
        ...
        """
        pass

    def __str__(self):
        pass


def HISTstockPrice(stock, period, interval):

    ticker = yf.Ticker(stock)
    data = ticker.history(period=period, interval=interval)  # 1-minute data for today
    df = pd.DataFrame(data)

    df["diff"] = df["Close"] - df["Open"]
    df["color"] = df["diff"].apply(lambda x: "green" if x >= 0 else "red")

    df.to_csv(f"{stock}.csv")
    return df

In [3]:
#####################
asset_name = "BTC-USD"
#####################
hist = HISTstockPrice(asset_name, "1y", "1d")

# implement rebalancing 50:50
Cash = 1000000  # dollars
port = Portfolio(Cash)
buyCount = 0
sellcount = 0
for i in range(hist.shape[0]):

    price = hist["Close"].iloc[i]
    # Fix: vvvvvvvvvvvvvvvvvv ทำเป็น  method: port.get_total_volume()
    total_vol = (
        sum(port.portfolio[asset_name]["volumes"])
        if asset_name in port.portfolio
        else 0
    )

    if asset_name not in port.portfolio:
        port.buy_bitcoin(price, 0.5 * port.cash, asset_name)
        buyCount += 1
    else:
        if (price * total_vol) / ((price * total_vol) + port.cash) > 0.5:
            dollars_amount = ((price * total_vol) - port.cash) / 2
            btc_amount = dollars_amount / price
            port.sell_bitcoin(
                price,
                btc_amount,  ############## fix
                asset_name,
            )
            sellcount += 1
        elif (price * total_vol) / ((price * total_vol) + port.cash) < 0.5:
            dollars_amount = abs(((price * total_vol) - port.cash) / 2)
            port.buy_bitcoin(price, dollars_amount, asset_name)
            buyCount += 1
        if port.cash < 10000:
            print("Cash is too low, stopping trading.")
            break

total_btc_value = hist["Close"].iloc[-1] * sum(port.portfolio[asset_name]["volumes"])

# Fix: vvvvvvvvvvvvvvvvvvvvvvvvv ใช้ method __str__
print(
    f"Total port's value: {port.cash + total_btc_value} USD ({port.cash}, {sum(port.portfolio[asset_name]['volumes']), total_btc_value, asset_name})"
)
print(f"Total buy count: {buyCount}, Total sell count: {sellcount}")

AttributeError: type object 'Asset' has no attribute 'name'

In [ ]:
#####################
asset_name = "BTC-USD"
#####################
hist = HISTstockPrice(asset_name, "1y", "1d")

# implement rebalancing 50:50
Cash = 1000000  # dollars
port2 = Portfolio(Cash)


def simulation(hist, port, asset_name):
    buyCount = 0
    sellcount = 0
    for i in range(hist.shape[0]):
        price = hist["Close"].iloc[i]
        total_vol = (
            sum(port.portfolio[asset_name]["volumes"])
            if asset_name in port.portfolio
            else 0
        )

        if asset_name not in port.portfolio:
            port.buy_bitcoin(price, 0.5 * port.cash, asset_name)
            buyCount += 1
        else:
            # print(price * total_vol, port.cash)
            # print(((price * total_vol) / (port.cash)) - 1)
            if ((price * total_vol) / (port.cash)) - 1 > 0.1:
                dollars_amount = ((price * total_vol) - port.cash) / 2
                btc_amount = dollars_amount / price
                port.sell_bitcoin(
                    price,
                    btc_amount,
                    asset_name,
                )
                sellcount += 1
            elif ((price * total_vol) / (port.cash)) - 1 < -0.05:
                dollars_amount = abs(((price * total_vol) - port.cash) / 2)
                port.buy_bitcoin(price, dollars_amount, asset_name)
                buyCount += 1
            if port.cash < 10000:
                print("Cash is too low, stopping trading.")
                break

    total_btc_value = hist["Close"].iloc[-1] * sum(
        port.portfolio[asset_name]["volumes"]
    )

    print(
        f"Total port's value: {port.cash + total_btc_value} USD ({port.cash}, {sum(port.portfolio[asset_name]['volumes']), total_btc_value, asset_name})"
    )
    print(f"Total buy count: {buyCount}, Total sell count: {sellcount}")


simulation(hist, port2, asset_name)